In [1]:
import pandas as pd
from config import config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf

import wandb
import tensorflow as tf
import numpy as np

# Loading, scaling the data
data has been explored and concaternated in `data_preparerationipynb`

In [2]:
# Load the dataset
df = pd.read_csv('../data/train.csv')
df.shape

(2000, 21)

In [3]:
# Separate the target variable from the features
X = df.drop('price_range', axis=1)

enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(df['price_range'].values.reshape(-1, 1))

#  Split the data into train, validation, and test sets 0.7, 0.15, 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=config['seed'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=config['seed'])

# Create scaling pipeline for the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\ns_a1_venv\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train), seed=config['seed']).batch(config['batch_size'])
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(config['val_batch_size'])
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(config['test_batch_size'])

In [5]:
input_shape = (20,)

# Build the model
input = tf.keras.layers.Dense(20, input_shape=input_shape, activation='relu'),
model = tf.keras.Sequential(input)

for size in config['hidden_layers']:
    model.add(tf.keras.layers.Dense(size, activation='relu'))
    model.add(tf.keras.layers.Dropout(config['dropout']))

output = tf.keras.layers.Dense(4, activation='softmax')
model.add(output)
#  Add the layers to the model

optimizer = tf.keras.optimizers.Adam(beta_1=config['beta1'], beta_2=config['beta2'], epsilon=config['epsilon'], learning_rate=config['lr'])
model.compile(optimizer=optimizer, loss=config['loss_keras'], metrics=['accuracy'])

In [6]:
name = 'TensorFlow L:{} DO:{} LR:{}'.format(str(config['hidden_layers']), config['dropout'], config['lr'])

wandb.init(project='phone-price-prediction', name=name, config=config)
wandb_callback = wandb.keras.WandbCallback()
model.fit(train_dataset, epochs=config['epochs'], validation_data=val_dataset, callbacks=[wandb_callback])
# build, compile, and train model here
test_loss, test_acc = model.evaluate(test_dataset)
wandb.log({'test_loss': test_loss, 'test_acc': test_acc})

# This is no longer needed
# y_pred = model.predict(X_val)
# y_pred_array = np.array(y_pred)
# y_val_array = np.array(y_val)

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xgazika (neural_networks_fiit). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50
36/44 [=======================>......] - ETA: 0s - loss: 1.4988 - accuracy: 0.2500

INFO:tensorflow:Assets written to: C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best)... Done. 0.0s


...

44/44 [==============================] - 1s 30ms/step - loss: 0.1894 - accuracy: 0.9293 - val_loss: 0.2117 - val_accuracy: 0.9133
Epoch 45/50
27/44 [=================>............] - ETA: 0s - loss: 0.1660 - accuracy: 0.9444

INFO:tensorflow:Assets written to: C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\gazik\Documents\FIIT_SS_2022\NS\A1\code\wandb\run-20230328_171550-dre9k3h0\files\model-best)... Done. 0.0s


44/44 [==============================] - 1s 27ms/step - loss: 0.1754 - accuracy: 0.9400 - val_loss: 0.2055 - val_accuracy: 0.9200
Epoch 46/50
44/44 [==============================] - 0s 3ms/step - loss: 0.1776 - accuracy: 0.9386 - val_loss: 0.2190 - val_accuracy: 0.9067
Epoch 47/50
44/44 [==============================] - 0s 3ms/step - loss: 0.1697 - accuracy: 0.9364 - val_loss: 0.2112 - val_accuracy: 0.9167
Epoch 48/50
44/44 [==============================] - 0s 3ms/step - loss: 0.1606 - accuracy: 0.9493 - val_loss: 0.2091 - val_accuracy: 0.9133
Epoch 49/50
44/44 [==============================] - 0s 3ms/step - loss: 0.1605 - accuracy: 0.9407 - val_loss: 0.2172 - val_accuracy: 0.9000
Epoch 50/50
5/5 [==============================] - 0s 3ms/step - loss: 0.1963 - accuracy: 0.9067


accuracy,▁▁▂▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▇▆▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁
val_accuracy,▁▂▃▄▄▄▅▅▆▆▆▇▇▇▇█████████████████████████
val_loss,██▇▇▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.94929
best_epoch,44
best_val_loss,0.20549
epoch,49
